In [ ]:
# analyseNearbyStars.ipynb
#
# Author: Alexander J Mustill, 2018
# Lund Observatory, Department of Astronomy & Theoretical Physics, 
# Lund University, Box 43, SE-221 00 Lund, Sweden
#
# If you make use of this code, please cite Mustill, Davies & Lindegren (2018, A&A submitted, https://arxiv.org/abs/1805.11638).
#
# Please direct any queries to Alexander Mustill, alex@astro.lu.se
#
# This queries the Gaia DR2 archive to find background objects to the list of nearby stars 
# obtained from getNearbyStars.ipynb, searching within 20 years' proper motion plus parallax 
# for any other sources. A list "interesting" of indices can be supplied if you have previously 
# down-selected a sample and don't want to run queries on all 700,000 objects.

In [ ]:
import datetime

import astropy.units as u
import numpy as np

from astropy.coordinates.sky_coordinate import SkyCoord
from astropy.units import Quantity
from astropy.io.votable import parse_single_table
from astropy.table import vstack

from gaiaDR2 import Gaia

from matplotlib import pyplot as plt

In [ ]:
vot_file = ['100pc_'+str(i).zfill(3)+'.vot' for i in range(360)]

In [ ]:
tmp = []
for i in range(len(vot_file)):
    tmp.append(parse_single_table(vot_file[i]).to_table())

s = vstack(tmp)


In [ ]:
deltaT = 20 #time we're interested in [yr]

Nclose = np.zeros(len(s)) #number of neighbours within search radius (INCLUDING SELF)
radius = []
results = []

In [ ]:
interesting = np.array([44, 1144, 1165, 1356, 2000, 2610, 2662, 3253, 3456, 3894, 4089, 
                        4123, 4357, 4757, 4817, 4862, 5035, 5182, 5335, 5565, 5606, 5658, 
                        5751, 6087, 6117, 6598, 6781, 6814, 7009, 7032, 7246, 8201, 8279, 
                        8288, 8294, 8354, 8412, 8556, 8697, 8725, 8871, 9513, 10670, 10750, 
                        10755, 10923, 11066, 11175, 11381, 11574, 11820, 11926, 11987, 11998, 
                        12287, 12379, 12433, 12668, 12842, 12876, 12887, 13637, 14114, 14115, 
                        14742, 14961, 15210, 15609, 15616, 15661, 15928, 16006, 16011, 16397, 
                        16563, 16586, 16788, 16789, 16934, 17196, 17846, 17881, 18081, 18236, 
                        18322, 18812, 18849, 18909, 19147, 19181, 19278, 19583, 20167, 20193, 
                        20762, 20808, 20812, 22187, 22895, 22934, 22980, 23067, 23463, 23487, 
                        23900, 23909, 24370, 24949, 25237, 25381, 25544, 25549, 25843, 25863, 
                        25889, 25911, 25989, 26003, 26237, 26461, 26464, 26597, 26772, 26948, 
                        27183, 27438, 27835, 27849, 28338, 28412, 28458, 28564, 28624, 28831, 
                        29019, 29149, 29431, 29667, 29863, 29865, 29996, 30019, 30044, 30153, 30550, 30551, 30573, 31090, 31292, 31345, 31505, 31566, 31671, 31679, 33016, 33155, 33248, 33252, 33257, 33391, 33442, 34418, 34707, 34708, 34830, 35370, 35803, 35944, 36123, 36558, 36762, 37119, 37236, 37237, 37350, 37396, 37751, 37876, 38808, 39126, 39418, 39684, 39734, 40636, 40759, 40814, 41817, 41915, 41986, 42530, 42558, 42647, 42685, 43532, 43599, 43686, 43719, 44170, 44184, 44199, 44294, 44424, 44738, 45109, 45172, 45663, 45901, 46021, 47357, 47358, 47567, 47618, 48402, 48579, 49409, 49654, 49901, 50624, 50946, 51087, 51430, 51536, 51598, 51693, 51805, 52343, 52486, 52503, 52624, 52853, 52979, 53431, 53501, 53750, 53921, 54416, 54483, 54756, 55034, 55270, 55312, 55317, 55626, 55851, 55855, 56276, 56337, 56609, 56816, 56926, 57291, 57348, 57542, 57633, 57805, 57815, 57840, 57854, 58057, 58191, 58198, 58274, 58369, 58459, 58576, 58619, 58975, 59286, 59953, 60086, 60312, 60801, 61124, 61585, 61831, 62627, 62751, 62942, 63138, 63144, 63207, 63242, 63552, 63553, 63707, 64118, 64167, 64434, 65156, 65324, 65450, 66064, 66391, 66491, 66700, 66703, 67048, 67364, 67487, 67712, 68502, 68565, 68608, 68675, 68687, 68757, 68857, 69186, 69465, 69475, 69497, 69537, 69895, 69902, 70367, 70465, 70906, 72490, 72496, 72723, 73022, 73029, 73446, 73533, 73553, 73629, 73764, 73777, 74158, 74399, 74420, 74540, 74882, 75413, 75444, 75446, 75781, 75802, 75805, 75862, 75891, 75950, 76083, 76352, 76358, 76371, 76407, 76633, 76656, 77431, 77476, 77750, 77789, 77858, 77909, 78121, 78174, 78607, 78745, 78921, 79004, 79059, 79228, 79609, 79705, 80116, 80175, 80178, 80203, 80332, 81307, 81815, 81839, 81970, 82062, 82538, 82580, 82811, 82930, 83408, 83627, 84049, 84081, 84251, 84269, 84523, 84543, 84579, 84691, 84704, 84705, 84860, 85139, 85208, 85376, 85447, 85613, 85759, 85873, 85919, 85986, 86014, 86076, 86086, 86194, 86258, 86286, 86360, 86468, 86884, 87044, 87050, 87078, 87458, 87736, 88065, 88367, 88396, 88423, 88766, 88838, 88898, 89055, 89275, 89487, 89675, 89676, 90195, 90196, 90685, 90758, 90955, 91642, 91679, 91764, 92001, 92044, 92126, 92210, 92323, 92331, 92370, 92655, 92695, 92856, 93010, 93141, 93214, 93352, 93377, 93418, 93714, 93752, 93861, 94077, 94393, 94626, 94676, 94772, 95012, 95377, 95398, 95459, 95594, 95766, 96093, 96227, 96886, 96933, 96956, 97081, 97244, 97802, 97806, 97829, 98234, 98266, 98297, 98298, 98887, 99071, 99295, 99674, 99884, 100284, 100700, 100771, 100919, 101008, 101380, 101425, 101482, 101918, 102022, 102094, 102241, 102371, 102611, 102950, 102983, 102990, 103710, 103940, 104081, 104509, 105077, 105829, 106300, 106334, 106420, 106707, 106759, 107188, 107341, 108599, 108903, 109047, 109194, 109283, 109302, 109312, 109343, 109355, 110098, 110123, 111055, 111133, 111592, 111750, 111830, 112356, 112373, 112780, 112895, 112938, 112968, 113346, 114082, 114097, 114908, 114923, 115117, 115464, 115780, 115894, 116762, 116801, 116918, 117942, 117999, 118411, 119424, 119526, 119711, 120162, 120181, 120395, 120498, 121117, 121413, 121707, 121847, 121976, 121994, 122072, 122121, 122490, 122509, 122601, 123318, 124050, 124114, 124692, 124750, 125048, 125484, 125961, 125965, 127438, 127503, 127557, 127731, 127923, 128007, 129249, 129273, 129502, 129503, 129515, 129797, 130091, 130144, 130181, 130229, 130586, 130654, 130782, 130790, 130940, 131054, 131785, 132261, 133306, 133593, 134023, 134837, 134954, 135003, 135150, 135215, 135276, 135283, 135321, 135591, 135593, 136223, 136468, 136540, 136552, 136934, 137002, 137251, 137280, 137429, 137516, 137584, 137708, 138138, 138152, 138292, 138385, 138435, 138478, 138744, 138923, 139210, 139359, 139383, 139465, 139501, 139596, 139892, 139908, 140269, 140403, 140523, 140622, 140881, 141153, 141294, 141303, 141676, 141778, 141801, 141880, 142234, 142358, 142369, 142707, 142801, 142825, 142984, 143065, 143235, 143588, 143589, 143815, 143901, 144912, 145170, 145321, 145327, 145402, 145421, 145563, 145671, 146153, 146423, 146749, 147004, 147102, 147266, 147476, 147546, 148089, 148366, 148495, 148507, 148859, 149167, 149371, 149428, 149563, 150696, 150903, 150973, 150998, 151232, 151652, 151734, 151845, 151940, 152519, 152719, 152740, 152775, 152825, 152847, 153040, 153103, 153585, 153653, 153707, 153829, 153832, 153992, 154455, 154567, 154616, 154961, 154971, 155163, 155220, 155365, 155645, 155692, 155748, 155815, 156141, 156142, 156277, 156366, 156450, 156597, 157455, 157546, 157849, 157982, 158867, 158930, 158975, 158985, 159353, 159360, 159570, 160278, 160354, 160418, 160438, 160778, 160897, 160928, 161040, 161236, 161303, 161462, 161584, 161714, 162024, 162192, 162484, 162498, 162714, 163414, 163445, 163548, 163749, 163803, 164301, 164329, 164385, 164456, 164630, 164651, 165260, 165775, 165896, 166046, 166138, 166201, 166221, 166574, 166580, 166581, 166922, 167001, 167411, 167424, 167533, 167647, 168188, 168381, 168432, 168554, 168564, 168822, 169429, 169474, 169490, 169868, 169909, 170390, 170391, 170510, 170615, 170946, 171482, 171535, 171602, 171688, 171948, 172024, 173257, 173366, 173421, 173472, 173565, 173616, 173732, 173827, 173893, 174137, 174200, 174378, 174700, 174961, 174991, 175194, 175330, 175515, 175682, 176006, 176060, 176094, 176326, 176401, 176710, 176926, 177017, 177043, 177512, 177515, 178110, 178153, 178534, 178717, 178826, 178876, 178953, 179135, 179171, 179761, 179949, 179951, 180146, 180649, 180650, 181166, 181200, 181299, 181311, 181395, 181550, 181807, 181808, 182062, 182295, 182432, 182449, 182657, 182790, 183033, 183174, 183178, 183267, 183497, 184287, 184542, 184722, 185273, 185301, 185612, 185764, 185845, 185932, 185967, 186339, 186503, 186751, 186884, 187475, 187678, 187835, 188371, 188436, 188499, 188923, 188930, 189120, 189392, 189416, 189539, 189701, 189945, 190235, 190334, 190575, 190673, 190684, 190877, 191220, 191589, 191756, 191820, 191885, 192222, 192464, 192702, 192803, 193004, 193241, 193773, 193785, 194045, 194370, 194371, 194434, 194885, 194919, 195264, 195656, 196094, 196215, 196804, 197811, 198190, 198527, 198985, 199102, 199402, 199469, 199517, 199745, 199882, 200466, 200467, 200765, 200769, 200772, 201154, 201412, 201671, 202855, 203014, 203149, 203323, 203419, 203524, 203789, 203792, 203875, 204025, 204222, 204567, 204725, 205456, 205504, 205566, 205572, 205894, 206350, 206478, 206538, 206567, 206589, 206621, 207149, 207396, 207844, 207918, 207919, 207926, 208100, 208145, 208515, 208734, 209143, 209317, 209770, 210165, 210385, 210467, 210846, 210880, 211468, 211650, 211921, 211922, 212116, 212701, 212771, 213088, 213171, 213303, 213551, 213814, 213954, 214177, 214638, 215122, 215254, 215281, 215741, 216212, 216646, 216785, 216849, 216850, 217274, 217565, 218367, 218561, 218790, 218959, 218976, 220025, 220528, 220736, 221054, 221258, 221328, 221654, 221810, 221910, 222073, 222105, 222803, 222923, 223314, 223566, 223593, 223784, 223979, 224014, 224093, 224294, 225422, 225741, 225879, 226093, 226301, 226346, 226492, 226553, 227048, 227291, 227691, 227793, 228004, 228254, 230198, 230209, 230290, 230842, 231767, 231954, 232129, 232458, 232697, 232816, 232891, 233089, 233398, 233639, 234067, 234176, 234356, 234363, 234374, 234380, 234766, 234934, 234936, 235081, 235622, 235639, 235969, 236424, 237875, 237982, 238385, 238487, 238961, 239224, 239303, 239329, 239353, 239722, 240362, 240420, 240421, 241999, 242210, 242282, 242325, 242337, 243355, 243437, 243441, 243444, 243572, 243703, 243863, 243893, 243894, 243944, 244422, 244429, 244497, 245177, 245366, 245693, 245997, 246229, 246373, 246529, 246567, 246587, 246630, 246756, 246767, 246913, 247052, 247107, 247132, 247195, 247347, 247353, 247404, 247405, 247813, 248088, 248185, 248336, 248374, 248536, 248549, 248563, 248594, 249063, 249350, 249421, 249479, 249831, 250319, 250383, 250877, 252208, 252389, 252390, 252408, 252544, 252910, 253322, 253381, 253605, 253732, 253960, 254551, 254604, 254666, 254744, 254835, 254892, 254937, 255003, 255163, 255283, 255338, 255529, 255539, 255541, 255564, 255601, 255806, 256260, 256419, 256516, 257074, 257328, 257348, 257456, 257832, 257836, 257883, 258136, 258774, 258847, 259309, 259337, 259487, 259626, 259723, 259869, 259941, 260197, 260618, 260801, 260968, 261028, 261110, 261168, 261184, 261263, 261353, 261878, 262429, 262577, 263067, 263465, 263667, 263756, 264120, 264135, 264216, 264302, 264544, 264762, 264932, 264938, 265054, 265264, 265555, 265678, 265951, 265962, 266103, 266307, 266654, 266787, 266804, 266940, 266980, 267105, 267142, 267232, 267596, 267600, 267924, 268044, 268072, 268078, 268095, 268180, 268450, 269335, 269566, 269783, 269854, 270624, 270652, 270656, 270984, 271025, 271229, 271408, 271576, 272142, 272418, 272494, 272622, 272663, 272759, 272982, 273447, 273699, 274026, 274449, 275306, 275550, 275646, 275679, 275857, 275985, 276052, 276120, 276190, 276199, 276202, 276462, 276556, 276716, 276721, 276986, 277115, 277153, 277527, 277687, 277723, 277781, 277821, 277948, 278149, 278258, 278408, 278789, 278967, 279008, 279461, 279916, 280195, 280470, 280813, 280904, 280936, 281189, 281615, 282025, 282184, 282295, 282396, 282803, 283177, 283298, 283487, 283488, 283840, 284151, 284220, 284502, 284648, 284658, 285147, 285292, 285384, 285472, 285654, 285687, 286100, 286279, 286299, 286971, 287012, 287037, 287144, 287221, 287341, 287514, 287765, 287876, 287904, 288513, 288639, 288920, 288935, 289122, 290167, 290230, 290609, 290621, 290681, 290825, 291075, 291253, 291376, 291392, 291920, 292139, 292219, 292766, 293150, 293251, 293597, 293601, 293704, 293807, 294055, 294114, 294326, 294720, 294909, 295138, 295157, 295391, 296656, 296718, 296900, 297394, 297467, 297475, 297532, 297997, 298207, 299059, 299219, 299230, 299538, 299691, 300057, 300548, 300643, 300675, 300828, 300952, 301077, 301871, 302043, 302079, 302244, 302972, 303305, 303348, 304380, 304622, 306121, 306199, 306422, 306835, 306916, 306964, 307104, 307626, 307689, 307697, 307737, 307901, 308005, 308740, 309172, 309495, 309909, 310103, 310392, 310789, 310971, 311478, 311488, 311614, 311792, 311910, 311942, 312341, 312750, 313021, 316475, 316925, 318866, 319249, 319339, 319438, 319505, 319551, 319690, 319795, 319825, 320002, 320157, 320158, 320161, 320236, 320374, 320730, 320998, 321967, 322600, 322976, 323103, 323150, 323288, 323431, 323623, 323624, 323633, 323746, 323841, 324000, 324331, 324524, 325143, 325297, 325746, 325789, 326382, 326420, 326424, 326425, 327314, 327601, 327677, 327741, 327842, 327854, 328188, 328208, 328322, 328704, 328726, 329229, 329243, 329605, 329628, 329896, 329955, 329986, 330152, 330195, 331148, 331293, 331348, 331491, 331548, 331970, 332575, 332602, 333074, 333230, 333419, 333604, 334004, 334033, 334431, 334528, 334676, 334679, 334950, 335174, 335279, 335761, 335857, 336481, 337690, 337780, 337992, 339227, 339452, 339664, 339846, 340079, 340091, 340935, 340946, 341215, 341262, 341482, 341708, 342185, 342501, 342585, 342666, 343119, 343357, 343880, 343919, 344654, 345681, 345908, 346550, 347500, 347787, 348165, 348201, 348236, 348417, 348498, 349277, 349280, 349406, 350213, 350772, 351316, 351397, 351844, 352881, 353304, 353566, 353863, 354236, 354598, 354781, 356152, 356171, 357357, 357884, 357953, 358600, 359425, 361148, 361212, 361559, 361621, 361815, 362387, 362420, 362468, 362656, 362809, 365222, 365520, 367260, 368978, 369302, 369586, 369601, 369721, 369817, 370235, 372051, 372325, 372478, 372768, 373450, 373725, 374817, 376150, 376261, 376266, 376299, 376312, 377646, 377701, 377901, 378100, 378184, 378366, 378922, 379170, 379214, 379421, 379500, 379686, 379966, 380238, 380374, 380600, 380645, 380732, 382027, 382068, 382318, 382332, 382824, 383058, 383073, 383390, 383663, 383688, 383712, 383945, 384044, 384064, 384514, 385061, 385354, 385973, 386227, 386316, 386571, 386612, 387672, 387723, 387773, 387801, 387823, 387908, 388268, 388416, 389143, 390221, 390418, 390662, 390881, 390905, 391052, 391598, 391737, 392078, 392230, 392334, 394306, 394358, 394359, 394788, 395720, 395944, 397253, 398221, 398704, 398706, 398996, 399383, 399546, 399824, 399913, 399969, 399971, 400035, 400254, 400337, 401385, 401711, 401865, 402316, 402610, 403034, 403145, 403750, 403904, 404161, 404168, 405384, 405739, 405771, 406783, 407214, 407398, 407407, 409186, 409391, 410099, 411622, 411632, 411719, 412462, 413576, 415535, 416725, 417052, 417145, 417496, 418189, 418221, 418239, 419122, 420447, 421757, 422359, 423516, 423850, 424183, 425011, 427135, 427610, 427944, 427985, 430103, 431007, 431507, 431695, 432501, 432965, 433347, 434169, 434395, 434397, 434671, 439711, 440314, 441912, 442117, 442625, 443911, 444384, 445880, 447077, 447186, 447529, 448190, 449635, 451328, 451921, 452215, 452660, 452771, 453306, 453784, 454238, 455166, 455546, 456489, 456691, 458711, 459533, 460220, 460489, 460506, 460793, 460975, 461186, 461776, 461777, 461823, 461978, 461999, 462355, 462931, 462940, 464148, 464727, 464986, 466776, 467394, 467897, 468611, 468704, 469100, 469946, 470485, 470569, 471222, 472640, 475026, 475667, 475936, 477269, 479010, 479342, 480064, 480564, 480573, 481523, 481809, 482026, 482190, 482500, 482868, 484150, 484234, 484901, 485212, 486079, 486510, 487656, 488068, 488077, 488403, 488492, 488494, 488502, 488658, 488676, 488882, 490251, 491316, 491368, 492087, 492433, 492952, 493459, 493548, 493583, 493629, 493845, 494166, 494261, 495205, 497172, 497594, 497859, 497862, 499022, 499657, 500507, 500762, 500824, 501211, 501493, 501496, 501599, 502932, 505039, 505050, 505129, 505815, 505816, 506175, 506228, 506637, 507418, 507997, 508147, 508236, 508559, 509126, 509185, 509320, 509541, 509661, 509947, 510916, 510929, 510945, 511716, 512191, 512229, 512537, 512596, 512599, 512753, 512985, 513095, 513297, 513638, 513670, 513848, 513932, 514300, 514317, 514771, 515097, 515301, 515913, 516004, 516044, 516063, 516072, 516149, 516153, 516637, 517387, 517835, 517907, 518292, 518429, 518454, 518467, 518498, 518602, 518806, 519285, 519324, 520111, 520309, 520383, 520848, 520908, 521139, 521148, 521226, 521775, 521776, 521943, 522017, 522456, 522799, 522867, 523068, 523219, 523269, 523796, 523863, 524262, 524289, 524840, 524970, 525980, 525991, 526426, 526550, 526769, 527050, 527237, 527246, 527271, 527673, 527674, 527785, 528126, 528403, 528679, 528779, 529099, 529412, 529443, 529473, 529950, 530013, 530015, 530277, 531828, 532260, 532472, 533357, 533743, 534149, 534289, 534377, 535187, 535440, 536084, 536105, 536652, 536918, 538562, 539336, 539394, 540037, 540538, 540544, 540577, 540964, 541014, 541284, 541445, 541470, 541582, 542764, 543236, 543613, 544237, 544277, 544838, 545833, 546034, 546414, 546442, 547407, 547584, 547979, 548042, 548260, 548305, 548948, 548949, 549113, 549287, 549387, 549409, 549762, 549849, 550272, 550277, 550940, 551237, 551437, 551732, 551998, 552026, 552384, 553134, 553660, 554075, 554491, 555464, 555632, 556210, 556952, 557167, 557201, 557833, 558360, 558900, 558936, 558947, 559211, 559425, 559496, 559515, 559744, 559971, 560181, 560263, 561586, 561993, 562079, 562173, 562330, 562496, 562684, 563198, 563470, 564508, 564611, 565135, 565339, 565416, 565484, 566171, 566925, 567398, 568087, 568517, 568562, 568732, 568960, 568982, 569988, 570098, 570338, 570912, 571016, 571101, 572090, 572238, 573275, 573407, 573996, 574042, 574443, 574579, 575674, 575956, 577721, 578075, 578679, 578759, 579377, 579821, 580264, 580540, 580874, 580892, 580893, 581009, 581253, 582054, 582322, 582537, 583100, 583210, 584955, 585506, 586263, 587946, 588032, 588057, 588065, 589155, 589565, 590356, 590473, 590664, 590833, 591270, 592019, 592314, 593507, 594200, 594424, 594853, 595158, 595657, 596049, 597080, 597081, 597327, 597380, 597976, 598165, 599143, 600768, 600788, 602127, 603018, 603923, 604468, 604605, 604796, 605268, 605584, 605592, 605612, 605666, 605913, 607372, 608530, 609794, 609882, 609975, 610768, 611158, 611404, 611783, 612072, 612077, 612744, 613868, 614036, 614057, 614575, 615275, 615506, 616207, 616394, 616468, 616559, 616560, 617057, 617390, 618793, 619771, 620259, 621077, 622143, 622254, 622567, 623051, 623080, 623547, 623766, 624079, 624225, 624807, 624904, 625142, 625237, 625554, 625698, 625882, 625993, 626564, 626966, 627199, 627278, 627345, 627569, 627618, 627661, 627718, 628008, 628167, 628219, 628258, 628265, 628283, 628326, 629576, 629691, 629723, 630048, 630208, 630467, 630859, 631328, 631371, 631958, 632574, 632706, 632739, 632787, 633151, 633575, 634080, 634103, 634238, 634266, 634709, 634857, 635088, 635186, 636082, 636161, 636855, 637055, 637075, 637077, 637264, 638200, 638227, 638537, 638592, 638809, 639007, 639012, 639219, 639451, 640009, 640237, 640381, 640407, 640845, 640964, 641132, 641205, 641432, 641740, 641770, 641817, 641979, 641989, 642210, 642339, 642484, 643169, 643196, 643203, 643593, 643630, 643872, 644140, 644303, 644745, 644781, 644831, 645095, 645240, 645322, 645425, 645563, 645641, 645703, 645713, 645717, 645723, 645878, 646178, 646196, 646302, 646315, 646388, 646512, 646621, 646770, 646861, 647071, 647235, 647358, 647604, 647670, 647944, 648006, 648196, 648553, 648722, 648825, 649262, 649354, 649565, 649865, 649972, 650106, 650164, 650200, 650292, 650308, 650473, 650979, 650998, 651042, 651312, 651609, 651760, 651877, 652997, 653177, 653252, 653266, 653269, 653545, 653546, 653996, 654031, 654221, 654695, 655021, 655192, 655365, 655393, 655465, 655518, 655576, 655632, 655668, 655753, 655758, 655768, 655887, 655996, 656033, 656327, 656630, 656633, 656645, 656707, 656749, 657255, 657263, 657290, 657489, 657684, 657884, 658090, 658470, 658479, 658668, 658669, 658673, 658691, 658698, 658718, 658782, 658877, 659015, 659270, 659493, 659673, 659755, 660159, 660279, 660559, 660654, 661014, 661052, 661305, 661319, 661419, 661427, 661995, 662095, 662108, 662400, 662552, 662634, 662868, 662894, 662918, 662919, 663133, 663205, 663395, 663572, 663588, 663600, 664024, 664098, 664138, 664139, 664308, 664498, 664588, 664656, 664684, 664685, 664730, 665130, 665144, 665262, 665434, 665524, 665580, 665631, 665815, 665830, 665846, 666229, 666317, 666910, 667035, 667061, 667230, 667307, 667356, 667421, 667573, 667723, 667842, 667948, 667959, 668184, 668214, 668389, 668747, 668927, 669384, 669385, 669602, 669972, 670046, 670068, 670179, 670248, 670635, 671213, 671330, 671560, 671735, 671742, 671906, 671929, 672052, 672431, 672748, 672991, 672994, 673092, 673335, 673447, 673654, 673766, 673987, 674223, 674330, 674670, 674925, 674944, 675213, 675473, 675976, 676120, 676438, 676483, 676720, 676862, 676876, 677062, 677244, 677281, 677726, 677745, 677802, 677847, 678181, 678979, 679032, 679110, 679595, 679893, 680019, 680476, 680786, 681096, 681325, 681327, 681671, 681672, 681976, 682321, 682507, 683051, 683137, 683218, 683230, 683244, 683397, 683437, 683909, 683927, 684333, 684338, 684726, 685707, 685764, 685800, 685908, 686074, 686202, 686212, 686326, 686619, 687142, 687161, 687289, 687310, 687328, 687374, 687432, 687471, 687516, 687770, 687779, 687890, 688117, 688191, 688194, 688199, 688241, 688323, 688481, 688660, 688993, 688994, 689142, 689341, 689377, 689447, 689585, 689586, 690313, 690442, 690485, 690916, 692026, 692144, 692260, 692409, 692483, 692584, 692900, 693422, 693486, 693943, 694247, 694672, 694876, 695182, 695673, 695787, 696338, 696426, 696655, 697228, 697325, 697394, 697642, 698256, 698401, 698643, 698851, 699038, 699379, 699422, 699580, 699890, 700053])

In [ ]:
#define some quality flags

ast_good = [False]*len(s)
phot_good = [False]*len(s)

ngood = 0
for i in range(len(s)):
    ast_good[i] = (np.sqrt(s['astrometric_chi2_al'][i]/(s['astrometric_n_good_obs_al'][i]-5)) 
                   < 1.2*max([1,np.exp(-0.2*(s['phot_g_mean_mag'][i]-19.5))]))
    phot_good[i] = ((s['phot_bp_rp_excess_factor'][i] > 1.0 + 0.015*(s['phot_bp_mean_mag'][i]-
                                                                     s['phot_rp_mean_mag'][i])**2) and 
                   (s['phot_bp_rp_excess_factor'][i] < 1.0 + 0.06*(s['phot_bp_mean_mag'][i]-
                                                                   s['phot_rp_mean_mag'][i])**2))
    if ast_good[i] and phot_good[i]:
        ngood += 1
#print(ast_good)
#print(phot_good)
#print(s['phot_bp_rp_excess_factor'])
print(str(ngood) + ' good stars of ' + str(len(s)))

In [ ]:
t1 = datetime.datetime.now()

#for i in range(len(s)):
start = 0

for i in range(len(s))[start:]:

#    uncomment the following if you have pre-generated the list of interesting stars
    if i in interesting:
#    uncomment the following if you want to check everything (eg first time on dataset)
#    if True:
#    uncomment the following to only allow objects passing the astrometric and photometric vetting
#    if ast_good[i] and phot_good[i]:
        

#compute total PM (cos delta conversion is already performed in the datafiles)
        pm = np.sqrt(s[i]['pmra']**2 + s[i]['pmdec']**2) / (1000*3600) # deg/yr

#search radius = annual parallactic motion plus PM over timescale of interest
        rad = Quantity(pm*deltaT + 2*s[i]['parallax']/(1000*3600),unit=u.deg)
        print('i = ',i)
        job = Gaia.launch_job_async("SELECT DISTANCE(POINT('ICRS',ra,dec), \
        POINT('ICRS',"+str(s[i]['ra'])+","+str(s[i]['dec'])+")) AS dist, * \
        FROM gaiadr2.gaia_source WHERE CONTAINS(POINT('ICRS',ra,dec),\
        CIRCLE('ICRS',"+str(s[i]['ra'])+","+str(s[i]['dec'])+","+str(rad.value)+"))=1 ORDER BY dist ASC")
        r = job.get_results()
    
        Nclose[i] = len(r)
        
        radius.append(rad)
        results.append(r)
    else:
        Nclose[i] = 1
        radius.append(None)
        results.append(None)
    
t2 = datetime.datetime.now()

print('Time elapsed: ',t2-t1)

In [ ]:
print(sum([i > 1 for i in Nclose]))


for i in range(len(s))[start:]:

    if Nclose[i] > 1:
    
        print('i = ',i,' Gaia DR2 ID: ',s[i]['source_id'],' at RA = ',s[i]['ra'],
              ' Dec = ',s[i]['dec'])
        print('Search radius: ',radius[i-start].to(u.arcsec))
        results[i-start]['dist','source_id','ra','dec'].pprint(max_width=-1,max_lines=-1)
        print('----------------------------------------------------------------------------------------------')
        print('')

In [ ]:
plt.hist(1000.0/s['parallax'])

plt.show()

In [ ]:
index = [np.where(Nclose == i) for i in range(int(max(Nclose)+1))]
[len(index[i][0]) for i in range(int(max(Nclose)+1))]

In [ ]:
len([i for i in range(len(s)) if Nclose[i] > 1])

In [ ]:
theta_E = 1.0 *u.mas * np.sqrt(s['parallax']/0.125)
r_E = theta_E * (1000/(s['parallax']*u.mas))/1000 * u.au

In [ ]:
plt.hist(theta_E.value)
plt.yscale('log')
plt.xlabel('Einstein radius [mas]')
plt.ylabel('# interesting stars')
plt.show()

In [ ]:
plt.hist(r_E.value)
plt.yscale('linear')
plt.xlabel('Einstein radius [au]')
plt.ylabel('# interesting stars')
plt.show()

In [ ]:
plt.scatter(s['parallax'],Nclose)

plt.xscale('log')
plt.yscale('log')

plt.xlabel('parallax [mas]')
plt.ylabel('close stars')

plt.show()

In [ ]:
time = np.linspace(0,deltaT,num=1001)

# only interested in alignments from this year
# this will also hopefully cut out the annoying duplicated sources
tStart = 3.0
earlyIndex = time < tStart
lateIndex = time >= tStart

ra = [None]*len(s)
dec = [None]*len(s)
dRASource = [None]*len(s)
dDecSource = [None]*len(s)
dRALens = [None]*len(s)
dDecLens = [None]*len(s)
Dmin = [None]*len(s)
Dmin2015 = [None]*len(s)

HasAClose = [False]*len(s)

for i in range(len(s)):

    ra[i] = s[i]['ra']*u.deg + s[i]['pmra']*time*u.mas/np.cos(np.radians(s[i]['dec']))
    dec[i] = s[i]['dec']*u.deg + s[i]['pmdec']*time*u.mas

    if Nclose[i] > 1:
        dRASource[i] = (results[i]['ra']-ra[i][0])*np.cos(np.radians(results[i]['dec']))*3600
        dDecSource[i] = (results[i]['dec']-dec[i][0])*3600

        dRALens[i] = (ra[i]-ra[i][0])*np.cos(np.radians(dec[i]))*3600
        dDecLens[i] = (dec[i]-dec[i][0])*3600

#Euclidean distance (neglect curvature for now)
        Dmin[i] = np.zeros(len(results[i]))
        Dmin2015[i] = np.zeros(len(results[i]))

        for j in range(len(results[i])):
            distance2 = (dRASource[i][j]-dRALens[i].value)**2 + (dDecSource[i][j]-dDecLens[i].value)**2
            
            distance2015 = np.sqrt(np.min(distance2[earlyIndex]))
            distance = np.sqrt(np.min(distance2[lateIndex]))
            Dmin2015[i][j] = distance2015
            Dmin[i][j] = distance
            if HasAClose[i] == False:
                if Dmin[i][j] < 1 and Dmin[i][j] < Dmin2015[i][j]:
                    HasAClose[i] = True

        print(i,Dmin[i])

In [ ]:
# this is a list of lenses that approach within 1 arcsec of a potential background source

for i in range(len(s)):
    if HasAClose[i]:
        print(i,s[i]['source_id'],s[i]['phot_g_mean_mag'],min(Dmin[i][1:]))

In [ ]:
# many of these will be physical binaries, so we need to cut out objects with similar parallax and pm
#(within 10%)

interesting2 = []

#uncomment these if you don't want to cut on them
pmrarel = 1
pmdecrel = 1

for i in interesting:
    good = 0
    for j in range(len(results[i])):
        prel = abs((s[i]['parallax'] - results[i][j]['parallax'])/s[i]['parallax'])
#        pmrarel = abs((s[i]['pmra'] - results[i][j]['pmra'])/s[i]['pmra'])
#        pmdecrel = abs((s[i]['pmdec']-results[i][j]['pmdec'])/s[i]['pmdec'])
        if (good == 0 and Dmin[i][j] < 1 and Dmin[i][j] < Dmin2015[i][j] and 
            prel > 0.1 and pmrarel > 0.1 and pmdecrel > 0.1):
            interesting2.append(i)
            good = 1
        
print(len(interesting2))
print(interesting2)

In [ ]:
i = 2610

print(i,'Gaia DR2 ',s[i]['source_id'],'N sources: ',len(results[i]))
print('Source               G mag                 Dmin')
print(s[i]['source_id'],s[i]['phot_g_mean_mag'],' 0.0')
for j in range(len(results[i]))[1:]:
    print(results[i][j]['source_id'],results[i][j]['phot_g_mean_mag'],Dmin[i][j])
print(' ')
print(' ')

plt.figure(figsize=[6,5])
plt.scatter(-dRASource[i],dDecSource[i],c=Dmin[i])
plt.plot(-dRALens[i],dDecLens[i])

plt.colorbar(label='min distance [arcsec]')

plt.xlabel("-Delta RA*cos(Dec) [arcsec]")
plt.ylabel("Delta Dec [arcsec]")
plt.axis('equal')

plt.show()